# Plot forward model

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from pcntoolkit.normative import estimate, predict, evaluate
from pcntoolkit.util.utils import create_design_matrix
from models import CNorm, CT, BNorm
from plotter import Plotter
from utils import reshape_data


atlas_to_use = 'glasser'

percentiles = [[.25, .75], [.05, .95], [.01, .99]]

sns.set_theme(
    context='paper',
    style='ticks',
    palette='tab10',
    font='helvetica',
    font_scale=1.5,
    color_codes=True,
    rc=None,
)

proj_dir = '/Users/philis/Documents/projects/b-norm_modelling' # exchange this with your local path
os.chdir(proj_dir) # we want to make sure we're in the correct project directory 

## CODE

In [2]:
# load the data
df_2yr = pd.read_csv(f'data/{atlas_to_use}/combined_features_2yr.csv')
df_4yr = pd.read_csv(f'data/{atlas_to_use}/combined_features_4yr.csv')

site_ids = df_2yr.site.unique().tolist()

ROI_lbls = df_2yr.columns[15:]

## plot c-norm normative curves

In [ ]:
os.chdir(proj_dir)
model_id = 'CNorm'
model = CNorm(ROI_lbls)
model.set_output_directories(proj_dir, atlas_to_use)

plotter = Plotter(model)

# we need this to compute the min and max age values at baseline across
# all ages in our participant pool
train_test_combined = pd.concat([train_data, test_data], axis=0)

xmin = train_test_combined[["age_bl", "age_2yr", "age_4yr"]].melt()["value"].min()
xmax = train_test_combined[["age_bl", "age_2yr", "age_4yr"]].melt()["value"].max()

print('xmin', xmin, 'xmax', xmax)

In [ ]:
# set this to true if you want to include the training data in the normative plot
incl_train = True
# set this to false if you want to plot the data for those participants with 2- and 4-year follow-up data
plot_2year_only = False
use_bspline = False
dataset = "sexspecific" # "full", "sexspecific"
markers = ['o', 'D']
colors = [[255 / 255, 193 / 255, 7 / 255], [12 / 255, 123 / 255, 220 / 255],
          [216 / 255, 27 / 255, 96 / 255]],  # (bl, 2yr, 4yr)

sexes = ["male", "female"]

####### DO NOT MAKE CHANGES HERE #######
for_save_name = '4yr'
timepoints = ["2yr", "4yr"]

suffix = "predict"

bspline_suffix = '_bspline' if use_bspline else ''

for r, roi in enumerate(ROI_lbls):

    # set up the subplots for the current ROI
    fig, axes = plt.subplots(2, 1, figsize=(5, 9), sharex=True, sharey=True)

    # make sure, we're in the project directory
    os.chdir(proj_dir)
    print("Running roi: ", roi, ":")

    for sexctr, sex in enumerate([1, 2]):

        this_model_dir = model.model_output_dir + bspline_suffix

        if dataset == 'sexspecific':
            current_ds = sexes[sexctr]
            roi_dir = os.path.join(this_model_dir, current_ds, roi)
        else:
            current_ds = "full"
            roi_dir = os.path.join(this_model_dir, "full", roi)

        data_dir = os.path.join(model.data_dir, current_ds)
        os.chdir(roi_dir)

        # load the model
        model.load(".")

        model.setup_dummy_data(xmin, xmax, sex, site_ids, model_type='full' if dataset == 'full' else 'sexspecific')
        cov_dummy_fname = f"cov_dummy_mean_sex{sex}_bspline.txt" if use_bspline else f"cov_dummy_mean_sex{sex}.txt"
        cov_dummy_file = os.path.join(roi_dir,  cov_dummy_fname)
        print('Making predictions with dummy covariates (for visualisation)')
        yhat, s2 = predict(cov_dummy_file, alg='blr', respfile=None, model_path=os.path.join(roi_dir, "Models"),
                           outputsuffix='_dummy')

        # extract the different variance components to visualise
        beta, junk1, junk2 = model.nm.blr._parse_hyps(model.nm.blr.hyp, model.X_dummy)
        s2n = 1 / beta  # variation (aleatoric uncertainty)
        s2s = s2 - s2n  # modelling uncertainty (epistemic uncertainty)

        # then, we warp dummy predictions to create the plots
        med, pr_int = model.W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), model.warp_param)

        # add the centile curves
        plotter.add_centile_curves(model.X_dummy[:, 1], yhat, s2, s2s, axes[sexctr])

        # if you choose you can also include the training data in the normative plot
        # while this can become messy, it could be necessary to make claims
        if (incl_train):
            idcs_train = train_data[(train_data.sex_bl == sex)].reset_index(drop=True).index.values
            plotter.add_training_data(axes[sexctr], current_ds, roi, idcs_train, med, site_ids, timepoint="bl",
                                      color=[0, 99 / 255, 64 / 255], alpha=.5)

        for t, tpt in enumerate(timepoints):
            # load the true data points
            dat = test_data[["sex_bl", f"{roi}_{tpt}"]]
            X_te = np.loadtxt(os.path.join(data_dir, 'with_4yr', 'cov_files_test', f'{roi}_{tpt}{bspline_suffix}.txt'))
            y_te = np.loadtxt(os.path.join(data_dir, 'with_4yr', 'resp_files_test', f'{roi}_{tpt}.txt'))

            # plot the data points
            clr = "blue" if sex == 1 else "red"

            idcs = dat[(dat.sex_bl == sex)].reset_index(drop=True).index.values
            X_rows2use = X_te[idcs, :]
            y_to_use = y_te[idcs]

            legend_ctr = 0
            for sid, site in enumerate(site_ids):
                # plot the true test data points
                # first, we select the data points belonging to this particular site
                idx = np.where(X_rows2use[:, sid + len(model.covariateColumns) + 1] != 0)[0]
                if len(idx) == 0:
                    # print('No data for site', sid, site, 'skipping...')
                    continue

                # then directly adjust the data
                idx_dummy = np.bitwise_and(model.X_dummy[:, 1] > X_rows2use[idx, 1].min(),
                                           model.X_dummy[:, 1] < X_rows2use[idx, 1].max())
                y_te_rescaled = y_to_use[idx] - np.median(y_to_use[idx]) + np.median(med[idx_dummy])
                alpha_factor = .7 if (tpt[0] == "bl") else 1
                clr_idx = ['bl', '2yr', '4yr'].index(tpt)
                axes[sexctr].scatter(X_rows2use[idx, 1], y_te_rescaled, s=24, color=colors[0][clr_idx],
                                     alpha=alpha_factor, marker=markers[t],
                                     label=f"{tpt}" if legend_ctr == 0 else None, edgecolor="black")

                if legend_ctr == 0:
                    axes[sexctr].legend()
                    legend_ctr += 1

            # add the median
            axes[sexctr].plot(model.X_dummy[:, 1], med, "black", linestyle="--")

        this_sex = "male" if sex == 1 else "female"
        axes[sexctr].set_title(f"{this_sex}: {roi}", fontsize=15)
        axes[sexctr].set_xlabel("age (in months)")
        axes[sexctr].set_ylabel("thickness (at timepoint)")
        axes[sexctr].grid(visible=True, linewidth=1, alpha=.5)

    # save the output
    out_dir = os.path.join(model.plot_dir + bspline_suffix, "normative_plots", dataset)
    os.makedirs(out_dir, exist_ok=True)
    out_name = roi + "_" + for_save_name + "_with-train.pdf" if incl_train else roi + "_" + for_save_name + ".pdf"
    fig.tight_layout()
    plt.savefig(os.path.join(out_dir, out_name), transparent=False, dpi=300)
    plt.close()
    os.chdir(proj_dir)

os.system(f'say "done plotting"')

## plot b-norm normative curves

so, in the thickness-age-sex model, we have the following covariates:

thickness, age_t1, age_t2, sex

We therefore need to create a dummy matrix in which we have
- thickness (varying), age_t1 (constant), age_t2 (constant), sex (constant) + sites
- thickness (constant), age_t1 (varying), age_t2 (constant), sex (constant) + sites
- thickness (constant), age_t1 (constant), age_t2 (varying), sex (constant) + sites

Now, the thickness values change for each ROI, we can therefore create these only during the ROI-loop (or we should). <br>
Age_t1, age_t2, and sex can be predefined and then concatenated with the respective thickness. 
<br>

We also need to plot only those particpants who are close to the mean age, since this is what we create centiles for.

In [5]:
os.chdir(proj_dir)
model_id = 'BNorm' # make sure to remove "age" directories
model = BNorm(ROI_lbls)
model.set_output_directories(proj_dir, atlas_to_use)

plotter = Plotter(model)

In [7]:
#sns.set(style='whitegrid')

incl_data = True
# set this to true if you want to include the training data in the normative plot
incl_train = True
# set this to false if you want to plot the data for those participants with 2- and 4-year follow-up data
plot_2year_only = False
dataset = "sexspecific"  # "full", "sexspecific"
markers = ['o', 'D']
colors = [[255 / 255, 193 / 255, 7 / 255], [12 / 255, 123 / 255, 220 / 255],
          [216 / 255, 27 / 255, 96 / 255]],  # (bl, 2yr, 4yr)

use_bspline = False

bl_age = 120 # can be different baseline age

####### DO NOT MAKE CHANGES HERE #######
for_save_name = f'4yr_blage-{bl_age}'
timepoints = ["2yr", "4yr"]

suffix = "predict"

bspline_suffix = '_bspline' if use_bspline else ''

suffix = "predict"

sexes = ["male", "female"]

data_dir = os.path.join(model.data_dir, dataset)
n_dummies = 75

for r, roi in enumerate(ROI_lbls):
    fig, axes = plt.subplots(2, 1, figsize=(5, 9), sharex=True, sharey=True)

    # make sure, we're in the project directory
    os.chdir(proj_dir)
    print("Running roi: ", roi, ":")

    # I NEED TO GET THE MIN/MAX VALUES FROM THE BL
    xmin = train_data[f"{roi}_bl"].min()
    xmax = train_data[f"{roi}_bl"].max()

    for sexctr, sex in enumerate([1, 2]):

        this_model_dir = model.model_output_dir + bspline_suffix

        if dataset == 'sexspecific':
            current_ds = sexes[sexctr]
            roi_dir = os.path.join(this_model_dir, current_ds, roi)
        else:
            current_ds = "full"
            roi_dir = os.path.join(this_model_dir, "full", roi)

        data_dir = os.path.join(model.data_dir, current_ds)
        os.chdir(roi_dir)

        # load the model
        model.load(".")

        model.setup_dummy_data(xmin, xmax, sex, site_ids, model_type='full' if dataset == 'full' else 'sexspecific',
                               bl_age=bl_age, n_dummies=75)
        cov_dummy_fname = f"cov_dummy_mean_sex{sex}_blage{bl_age}_bspline.txt" if use_bspline else f"cov_dummy_mean_sex{sex}_blage{bl_age}.txt"
        cov_dummy_file = os.path.join(roi_dir, cov_dummy_fname)
        print('Making predictions with dummy covariates (for visualisation)')
        yhat, s2 = predict(cov_dummy_file, alg='blr', respfile=None, model_path=os.path.join(roi_dir, "Models"),
                           outputsuffix='_dummy')

        # extract the different variance components to visualise
        beta, junk1, junk2 = model.nm.blr._parse_hyps(model.nm.blr.hyp, model.X_dummy)
        s2n = 1 / beta  # variation (aleatoric uncertainty)
        s2s = s2 - s2n  # modelling uncertainty (epistemic uncertainty)

        # then, we warp dummy predictions to create the plots
        med, pr_int = model.W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), model.warp_param)

        # add the centile curves
        plotter.add_centile_curves(model.X_dummy[:, 1], yhat, s2, s2s, axes[sexctr])

        # if you choose you can also include the training data in the normative plot
        # while this can become messy, it could be necessary to make claims
        if (incl_train):
            train_sex = train_data[train_data.sex_bl == sex].reset_index(drop=True)
            idcs_train = train_sex[train_sex.age_bl == bl_age].index.values
            plotter.add_training_data(axes[sexctr], current_ds, roi, idcs_train, med, site_ids,
                                      timepoint="2yr", color=[0, 99 / 255, 64 / 255], alpha=.5)

        if (incl_data):
            for t, tpt in enumerate(timepoints):
                # load the true data points
                dat = test_data[["sex_bl", "age_bl", f"{roi}_{tpt}"]]
                X_te = np.loadtxt(os.path.join(data_dir, 'with_4yr', 'cov_files_test', f'{roi}_{tpt}{bspline_suffix}.txt'))
                y_te = np.loadtxt(os.path.join(data_dir, 'with_4yr', 'resp_files_test', f'{roi}_{tpt}.txt'))

                # plot the data points
                clr = "blue" if sex == 1 else "red"

                # help = tpt[0] if t > 0 else "2yr"
                dat_sex = dat[dat.sex_bl == sex].reset_index(drop=True)
                idcs = dat_sex[(dat_sex.age_bl == bl_age)].index.values
                print(roi, sex, bl_age, tpt, len(idcs))
                X_rows2use = X_te[idcs, :]
                y_to_use = y_te[idcs]

                # now get only those participants that fall in -/+2 months of the age
                second_tpt_age = model.X_dummy[0, 3] if tpt[0] == "2yr" else model.age_4yr_dummy[0]

                legend_ctr = 0
                for sid, site in enumerate(site_ids):
                    # plot the true test data points
                    # first, we select the data points belonging to this particular site
                    idx = np.where(X_rows2use[:, sid + len(model.covariateColumns) + 1] != 0)[0]
                    if len(idx) == 0:
                        # print('No data for site', sid, site, 'skipping...')
                        continue

                    # then directly adjust the data
                    idx_dummy = np.bitwise_and(model.X_dummy[:, 1] > X_rows2use[idx, 1].min(),
                                               model.X_dummy[:, 1] < X_rows2use[idx, 1].max())
                    y_te_rescaled = y_to_use[idx] - np.median(y_to_use[idx]) + np.median(med[idx_dummy])
                    alpha_factor = 1.0 if (tpt[0] == "2yr" or tpt[0] == "bl") else 1.0
                    clr_idx = ['bl', '2yr', '4yr'].index(tpt)
                    axes[sexctr].scatter(X_rows2use[idx, 1], y_te_rescaled, s=24, color=colors[0][clr_idx],
                                                 alpha=alpha_factor, marker=markers[t],
                                                 label=f"{tpt}" if legend_ctr == 0 else None, edgecolors='black')

                    if legend_ctr == 0:
                        legend_ctr += 1

        # add the median
        axes[sexctr].plot(model.X_dummy[:, 1], med, "black", linestyle="--")
        axes[sexctr].legend()
        this_sex = "male" if sex == 1 else "female"
        axes[sexctr].set_title(f"{this_sex}: {roi} (bl_age = {bl_age})", fontsize=15)
        axes[sexctr].set_xlabel("thickness at bl")
        axes[sexctr].set_ylabel("thickness (at timepoint)")
        axes[sexctr].grid(visible=True, linewidth=1, alpha=.5)

    # save the output
    out_dir = os.path.join(model.plot_dir, "normative_plots", dataset);
    os.makedirs(out_dir, exist_ok=True)
    out_name = roi + "_" + for_save_name + "_with-train.pdf" if incl_train else roi + "_" + for_save_name + ".pdf"
    fig.tight_layout()
    plt.savefig(os.path.join(out_dir, out_name), transparent=False, dpi=300)
    plt.close()
    os.chdir(proj_dir)

os.system(f'say "done plotting"');

Running roi:  lh_avg :
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
lh_avg 1 120 2yr 19
lh_avg 1 120 4yr 19
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
lh_avg 2 120 2yr 19
lh_avg 2 120 4yr 19


/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-

Running roi:  lh_L_V1 :
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
lh_L_V1 1 120 2yr 19
lh_L_V1 1 120 4yr 19
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
lh_L_V1 2 120 2yr 19
lh_L_V1 2 120 4yr 19


/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-

Running roi:  lh_L_MST :
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
lh_L_MST 1 120 2yr 19
lh_L_MST 1 120 4yr 19
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
lh_L_MST 2 120 2yr 19
lh_L_MST 2 120 4yr 19


...

/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-

Running roi:  rh_R_a32pr :
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
rh_R_a32pr 1 120 2yr 19
rh_R_a32pr 1 120 4yr 19
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
rh_R_a32pr 2 120 2yr 19
rh_R_a32pr 2 120 4yr 19


/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-

Running roi:  rh_R_p24 :
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
rh_R_p24 1 120 2yr 19
rh_R_p24 1 120 4yr 19
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
rh_R_p24 2 120 2yr 19
rh_R_p24 2 120 4yr 19


/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/philis/anaconda3/envs/norm312/lib/python3.12/site-